In [12]:
import pandas as pd

# Paths to the datasets
train_path = "train_data.txt"
test_path = "test_data_solution.txt"

# Load training data
movies = pd.read_csv(train_path, sep=':::', header=None, names=['title', 'genre', 'description'])

# Load testing data
movies_test = pd.read_csv(test_path, sep=':::', header=None, names=['title', 'genre', 'description'])


C:\Users\jhala\AppData\Local\Temp\ipykernel_22480\3888527641.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_csv(train_path, sep=':::', header=None, names=['title', 'genre', 'description'])
C:\Users\jhala\AppData\Local\Temp\ipykernel_22480\3888527641.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies_test = pd.read_csv(test_path, sep=':::', header=None, names=['title', 'genre', 'description'])


In [13]:
# Display the first few rows of the training data
print("Training data:")
print(movies.head())

# Display the first few rows of the testing data
print("\nTesting data:")
print(movies_test.head())

# Check for any missing values
print("\nTraining data missing values:")
print(movies.isnull().sum())

print("\nTesting data missing values:")
print(movies_test.isnull().sum())


Training data:
                                title       genre  \
1       Oscar et la dame rose (2009)       drama    
2                       Cupid (1997)    thriller    
3   Young, Wild and Wonderful (1980)       adult    
4              The Secret Sin (1915)       drama    
5             The Unrecovered (2007)       drama    

                                         description  
1   Listening in to a conversation between his do...  
2   A brother and sister with a past incestuous r...  
3   As the bus empties the students for their fie...  
4   To help their unemployed father make ends mee...  
5   The film's title refers not only to the un-re...  

Testing data:
                           title          genre  \
1          Edgar's Lunch (1998)       thriller    
2      La guerra de papá (1977)         comedy    
3   Off the Beaten Track (2010)    documentary    
4        Meu Amigo Hindu (2015)          drama    
5             Er nu zhai (1955)          drama    

              

In [14]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import re

space = re.compile('[/(){}\[\]\|@,;]')
symbols = re.compile('[^0-9a-z #+_]')
stopwords = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower() # lowercase text
    text = space.sub(' ', text) 
    text = symbols.sub('', text)  
    text = re.sub(r'[0-9]+','',text) #delete one or more digit ocurrences
    text = ' '.join(word for word in text.split() if word not in stopwords) # delete stopwords from text
    return text

# Apply preprocessing to description column
movies['description'] = movies['description'].apply(preprocess_text)
movies_test['description'] = movies_test['description'].apply(preprocess_text)


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(movies['description'], movies['genre'], test_size=0.2, random_state=42)

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Initialize Logistic Regression classifier
classifier = LogisticRegression(max_iter=1000)

# Create a pipeline
pipeline = Pipeline([
    ('tfidf', tfidf_vectorizer),
    ('clf', classifier)
])

# Fit the model
pipeline.fit(X_train, y_train)

# Predictions on the test set
y_pred = pipeline.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.58

Classification Report:
               precision    recall  f1-score   support

      action        0.51      0.26      0.35       263
       adult        0.76      0.22      0.34       112
   adventure        0.47      0.14      0.22       139
   animation        0.60      0.09      0.15       104
   biography        0.00      0.00      0.00        61
      comedy        0.51      0.58      0.55      1443
       crime        0.20      0.01      0.02       107
 documentary        0.67      0.84      0.75      2659
       drama        0.54      0.78      0.64      2697
      family        0.52      0.10      0.17       150
     fantasy        0.00      0.00      0.00        74
   game-show        0.94      0.42      0.59        40
     history        0.00      0.00      0.00        45
      horror        0.64      0.57      0.60       431
       music        0.65      0.49      0.56       144
     musical        0.50      0.02      0.04        50
     mystery        0.00 

c:\Users\jhala\OneDrive\Documents\GitHub\JHALAK_CODSOFT\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jhala\OneDrive\Documents\GitHub\JHALAK_CODSOFT\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jhala\OneDrive\Documents\GitHub\JHALAK_CODSOFT\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [16]:
# Predictions on the test set
y_pred_test = pipeline.predict(movies_test['description'])

# Append predictions to test dataset for review
movies_test['predicted_genre'] = y_pred_test

# Display the test dataset with predictions
print("Test data with predictions:")
print(movies_test.head())


Test data with predictions:
                           title          genre  \
1          Edgar's Lunch (1998)       thriller    
2      La guerra de papá (1977)         comedy    
3   Off the Beaten Track (2010)    documentary    
4        Meu Amigo Hindu (2015)          drama    
5             Er nu zhai (1955)          drama    

                                         description predicted_genre  
1  lr brane loves life car apartment job especial...          drama   
2  spain march quico naughty child three belongin...          drama   
3  one year life albin family shepherds north tra...    documentary   
4  father died hasnt spoken brother years serious...          drama   
5  known internationally martial arts superstar b...          drama   
